In [29]:
# !pip install selenium
# !pip install beautifulsoup4

In [30]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from tqdm import tqdm

In [31]:
#path to the webdriver file
service = webdriver.ChromeService(executable_path = 'chromedriver.exe')
driver = webdriver.Chrome(service=service)
url = 'https://www.imdb.com/search/title/?groups=top_100'
driver.get(url)


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=120.0.6099.130)
Stacktrace:
	GetHandleVerifier [0x00007FF6E2E32142+3514994]
	(No symbol) [0x00007FF6E2A50CE2]
	(No symbol) [0x00007FF6E28F76AA]
	(No symbol) [0x00007FF6E28D0AFD]
	(No symbol) [0x00007FF6E296CB1B]
	(No symbol) [0x00007FF6E298218F]
	(No symbol) [0x00007FF6E2965D93]
	(No symbol) [0x00007FF6E2934BDC]
	(No symbol) [0x00007FF6E2935C64]
	GetHandleVerifier [0x00007FF6E2E5E16B+3695259]
	GetHandleVerifier [0x00007FF6E2EB6737+4057191]
	GetHandleVerifier [0x00007FF6E2EAE4E3+4023827]
	GetHandleVerifier [0x00007FF6E2B804F9+689705]
	(No symbol) [0x00007FF6E2A5C048]
	(No symbol) [0x00007FF6E2A58044]
	(No symbol) [0x00007FF6E2A581C9]
	(No symbol) [0x00007FF6E2A488C4]
	BaseThreadInitThunk [0x00007FF9BBD57344+20]
	RtlUserThreadStart [0x00007FF9BDB226B1+33]


In [ ]:
# Set initial empty list for each element:
title = []
link = []
# Grab the block of each individual movie
# blocks = driver.find_elements_by_class_name('lister-item')
blocks = driver.find_elements(By.CLASS_NAME, "ipc-metadata-list-summary-item")

# Set up a for loop to run through all movies
for i in range(0,50):
    # Extracting title
    ftitle = blocks[i].find_element(By.CLASS_NAME, 'ipc-title__text').text

    # Then extract the link with cleaned title
    flink = blocks[i].find_element(By.LINK_TEXT, ftitle).get_attribute('href')

    # Add item to the respective lists
    title.append(ftitle)
    link.append(flink)


In [38]:
user_review_links = []
for url in link:
    try:
        #setup user agent for BS4, except some rare case, it would be the same for most browser 
        user_agent = {'User-agent': 'Mozilla/5.0'}
        #Use request.get to load the whole page
        response = requests.get(url, headers = user_agent)
        #Parse the requests object to BS4 to transform it into html structure
        soup = BeautifulSoup(response.text, 'html.parser')
        #Find the link marked by the USER REVIEWS link text.
        review_link = 'https://www.imdb.com'+soup.find('a', text = 'User reviews').get('href')
        #Append the newly grabed link into its list
        user_review_links.append(review_link)
    except:
            print(url)

https://www.imdb.com/title/tt0038650/?ref_=sr_t_2


In [1]:
len(user_review_links)

NameError: name 'user_review_links' is not defined

In [ ]:
movie_title = []
review_title = []
content = []
rating = []
index = 0

for review_link in user_review_links:

    review_driver = webdriver.Chrome(service=service)
    review_driver.get(review_link)

    fmovie_title = title[index]
    index += 1

    blocks = review_driver.find_elements(By.CLASS_NAME, "lister-item-content")

    for i in range(len(blocks)):
        try:
            # Extracting title
            ftitle = blocks[i].find_element(By.CLASS_NAME, 'title').text
            fcontent = blocks[i].find_element(By.CLASS_NAME, 'content').text
            fratting = blocks[i].find_element(By.CLASS_NAME, 'rating-other-user-rating').text

            # Add item to the respective lists
            movie_title.append(fmovie_title)
            review_title.append(ftitle)
            content.append(fcontent)
            rating.append(fratting)
        except:
            continue
    review_driver.quit()
    

# Export to csv file

In [20]:
#Build data dictionary for dataframe
data = {'Movie_name': movie_title, 
        'Review Rating': rating,
        'Review title': review_title, 
        'Review_content' : content
   }
#Build dataframe to export
review = pd.DataFrame(data = data)

In [21]:
review.head()

,Movie_name,Review Rating,Review title,Review_content
0,1. Oppenheimer,8/10,"A challenging watch to be sure, but a worthwhi...",
1,1. Oppenheimer,9/10,Murphy is exceptional,You'll have to have your wits about you and yo...
2,1. Oppenheimer,7/10,Quality but exhausting,I'm a big fan of Nolan's work so was really lo...
3,1. Oppenheimer,10/10,A brilliantly layered examination of a man thr...,"""Oppenheimer"" is a biographical thriller film ..."
4,1. Oppenheimer,10/10,Nolan delivers a powerfull biopic that shows t...,This movie is just... wow! I don't think I hav...


In [22]:
review.tail()

,Movie_name,Review Rating,Review title,Review_content
978,47. Star Wars: Episode IV - A New Hope,10/10,Just plain fun!,
979,47. Star Wars: Episode IV - A New Hope,6/10,A great movie - for its time,
980,47. Star Wars: Episode IV - A New Hope,10/10,A long time ago in a childhood not too far awa...,Princess Leia is captured and held hostage by ...
981,47. Star Wars: Episode IV - A New Hope,10/10,From the Lucasian Chair of Entertainment - a m...,I first saw this film on a warm March Sunday e...
982,47. Star Wars: Episode IV - A New Hope,10/10,the landmark space fantasy,what can you say about his movie.it set the ba...


In [24]:
review.to_csv('imdb_top_movie_review.csv',index=False)

In [34]:
link_data = {'Movie_name': title, 
        'Movie link': link,
        'Review link': user_review_links
   }
#Build dataframe to export
moive_review_link = pd.DataFrame(data = link_data)

ValueError: All arrays must be of the same length